In [1]:
import pickle
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from random import choice
from parse_utils import get_title, get_dates, get_doc_type, get_rubrics
import re

In [2]:
with open("docs.pkl", "rb") as f:
    docs = pickle.load(f)

In [3]:
# https://company.rzd.ru/ru/9353/page/105104?id=1489

In [4]:
for doc in docs:
    if doc["href"] == "https://company.rzd.ru/ru/9353/page/105104?id=1604":
        break

In [5]:
# doc = docs[0]
# doc = choice(docs)

html = doc["html"]
soup = BeautifulSoup(html, 'html.parser')

all_links = soup.find_all("a")
all_hrefs = [link.attrs['href'] for link in all_links]

print(doc["topic"])
print(doc['href'])
title = soup.find("h2", {"class": "card-header"}).text
title = re.sub(r"\s+", " ", title)
title

Деятельность ОАО «РЖД»
https://company.rzd.ru/ru/9353/page/105104?id=1604


'Коллективный договор открытого акционерного общества «Российские железные дороги» на 2023-2025 годы'

In [45]:
# find tags by class

date_elements = soup.find_all("div", {"class": "date-top"})

dates = []
for el in date_elements:
    date_type = el.find("span", {"class": "text-more-grey"}).text.strip()
    date = el.text.replace(date_type, "").strip()
    dates.append({
        "type": date_type,
        "date": date
    })

dates

[{'type': 'Дата официальной публикации', 'date': '17.03.2021'},
 {'type': 'Редакция от', 'date': '15.10.2020'}]

In [46]:
doc_type = soup.find_all("div", {"class": "doc-type"})[0].text
doc_type = doc_type.replace("\xa0", " ")
doc_type_date = re.search(r"(?<=от )\d\d\.\d\d\.\d\d\d\d", doc_type)
doc_type_date = None if doc_type_date is None else doc_type_date.group()
doc_type, doc_type_date

('Распоряжение ОАО «РЖД» от 01.09.2020 № 1871р', '01.09.2020')

In [47]:
# find elements by css selector

rubrics_a = soup.select(".rubrics > a")
rubrics = [r.text for r in rubrics_a]
rubrics

['Сотрудникам']

In [48]:
# Для удобства чтения 
for el in soup.find_all("div", {'class':'print-hide'}):
    el.decompose()

# Для удобства чтения 
for el in soup.find_all("a", {'class':'goto-rubrics'}):
    # print(el.text.strip())
    el.decompose()

In [49]:
# find elements with class "static-content" and ANY non empty id
# parts = soup.find_all("div", {"class": "static-content"})
elements = soup.select(".attached > .static-content")
elements = [elem for elem in elements if len(elem.text.replace(" ", "")) > 3] # get('id') and elem['id'].strip()

print("Elements:", len(elements))

if len(elements) == 0:
    # Ищем ссылку где есть ?action=download
    # links = soup.find_all("a", {"href": True})
    # hrefs = [link['href'] for link in all_links]
    hrefs = [href for href in all_hrefs if "action=download" in href]
    hrefs = ["https://company.rzd.ru" + href for href in hrefs]

    print("Hrefs:", hrefs)

else:
    attachments = []
    parts = []
    for part in elements:
        part_title = part.find("h3")
        if part_title is None:
            part_title = ""
        else:
            part_title = part_title.text.strip()

        contents = part.find_all("div", {"class": "onePartTextOut_text"})
        # print(len(contents))

        if part_title.lower() == 'приложения':
            attachments.append(part)
        else:
            parts.append(part)

    print("Len parts:", len(parts))
    print("Len attachments:", len(attachments))


Elements: 3
Len parts: 2
Len attachments: 1


In [27]:
# text=parts[0].get_text(separator = '\n', strip = True)
# print(text)

УТВЕРЖДАЮ
Вице-президент ОАО "РЖД"
В.Г.Лемешко
(С изменениями утв. ОАО "РЖД" от 05.05.2009 и Распоряжением ОАО "РЖД" от 01.12.2009 N 2443р)


In [50]:
print(parts[0].prettify())

<div class="static-content">
 <p>
  (в ред. Распоряжения ОАО «РЖД» от 15.10.2020 N 2288/р)
 </p>
 <p>
  В соответствии с пунктом 7.35 Коллективного договора ОАО «РЖД» на 2020-2022 годы:
 </p>
 <p>
  1. Утвердить прилагаемый Временный порядок начисления и обмена баллов бонусного пакета (далее - Порядок).
 </p>
 <p>
  2. Заместителям начальника по кадрам и социальным вопросам центральной дирекции моторвагонного подвижного состава Гудовой М.В., Северной железной дороги Жованик Н.А., Октябрьской железной дороги Середе С.И.:
  <br/>
  (в ред. Распоряжения ОАО «РЖД» от 15.10.2020 N 2288/р)
 </p>
 <p style="padding-left: 40px;">
  обеспечить ознакомление работников пилотного полигона с Порядком, утвержденным настоящим распоряжением;
 </p>
 <p style="padding-left: 40px;">
  назначить работников, ответственных за ведение работы по предоставлению бонусного пакета;
 </p>
 <p style="padding-left: 40px;">
  организовать работу по предоставлению бонусного пакета работникам пилотного полигона.
 </p>
